# Example Notebook for Text Generation Metric Evaluation

This notebook demonstrates example use cases for the Valor text generation metrics. The Valor text generation metrics can be used across a variety of tasks which typically, but not always, involve prompting an LLM to generate some text. Use cases include Query Answering, Retrieval Augmented Generation (which can be thought of as a subcase of Q&A), Summarization and Content Generation. 

Some of the metrics can be applied across different use cases. For example, the BLEU metric can be used to compare predictions (generated text) to groundtruth answers in the case of Q&A/RAG, and can also be used to compare predictions (generated text) to groundtruth summaries in the case of Summarization. Conversely, some of the metrics are specific to a use case, such as the ContextRecall metric for RAG or the Summarization score for Summarization. 

In all three use cases below, we generate answers using GPT3.5-turbo and evaluate those answers with a variety of metrics. For the text comparison metrics, we compare GPT3.5-turbo's responses to groundtruth Huggingface answers/summaries for the RAG and Summarization datasets. For the llm guided metrics (which include the RAG metrics, Summarization metrics and general text generation metrics), we are using GPT4o to evaluate the responses of GPT3.5-turbo. 

The first example is RAG for Q&A. We download a RAG dataset from HuggingFace, use Llama-Index and GPT3.5-turbo to generate answers, and evaluate those answers with text comparison metrics, RAG metrics and general text generation metrics.

The second example is Summarization. We download a CNN news dataset from HuggingFace which includes groundtruth summaries. We ask GPT3.5-turbo to summarize the articles. Then, we evaluate those summaries with text comparison metrics, summarization metrics and general text generation metrics.

The third example is content generation. We manually create a few queries, each of a different query type (creative, educational, professional). Then we evaluate the generated content with general text generation metrics. 

# Connect to Valor API

In [1]:
import os
import pandas as pd
from valor.enums import EvaluationStatus
from valor import Annotation, Datum, Dataset, Model, GroundTruth, Client, Prediction, connect

# Connect to Valor API.
connect("http://0.0.0.0:8000")
client = Client()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]

ClientConnectionFailed: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /api-version (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb3ab0a6450>: Failed to establish a new connection: [Errno 111] Connection refused'))

# Use Case #1: RAG for Q&A

## Download and Save the Corpus for the RAG Pipeline

In [ ]:
from datasets import load_dataset

In [ ]:
corpus_dataset = load_dataset("rag-datasets/mini_wikipedia", "text-corpus")["passages"]
print(corpus_dataset)

# For each passage in corpus_dataset, save that passage to a .txt file with the passage_id as the filename.
for passage in corpus_dataset:
    with open(f"./rag_corpus/{passage["id"]}.txt", "w") as f:
        f.write(passage["passage"])

## Load Queries and get Answers with Llama-Index

In [ ]:
import csv
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
# Load the query dataset. 
qa_dataset = load_dataset("rag-datasets/mini_wikipedia", "question-answer")["test"]
qa_dataset = qa_dataset.shuffle(seed=42)
print(qa_dataset)

In [ ]:
# Loads in the rag_corpus and builds an index.
# Initialize a query_engine, which will use GPT3.5-turbo by default with calls to OpenAI's API.
# You must specify your OpenAI API key in the environment variable OPENAI_API_KEY for the below code to function. 
documents = SimpleDirectoryReader("rag_corpus").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
# sample use
response = query_engine.query("What country borders Argentina and Brazil?")
print(response)
print(response.source_nodes)

response = query_engine.query("What color is a penguin?")
print(response)
print(response.source_nodes)

In [ ]:
if os.path.exists("rag_data.csv"):
    os.remove("rag_data.csv")

NUMBER_OF_RECORDS = 50

with open("rag_data.csv", mode="w") as data_file:
    data_writer = csv.writer(data_file, delimiter=",", quoting=csv.QUOTE_MINIMAL)
    data_writer.writerow(["query", "groundtruth", "prediction", "context_list"])

    for i in range(NUMBER_OF_RECORDS):
        query = qa_dataset[i]["question"]
        groundtruth = qa_dataset[i]["answer"]
        print(f"{i}: {query}")

        response_object = query_engine.query(query)
        response = response_object.response
        print(f"response: {response}")
        context_list = []
        for i in range(len(response_object.source_nodes)):
            context_list.append(response_object.source_nodes[i].text)
        data_writer.writerow([query, groundtruth, response, context_list])
    
    data_file.close()

## Evaluation in Valor

In this example, the RAG pipeline produces answers to the given queries by retrieving context and then generating answers based on the context and query. Groundtruth answers are also known for these queries. Both the datums (which contain the queries) and the groundtruths are added to the dataset. Then, the predictions are added to the model, which includes the answer and the context used to generate that answer. 

The metrics requested include some text comparison metrics (BLEU and ROUGE), which do a text comparison between the generated answer and the groundtruth answer for the same datum. If the user only desires these metrics, then they do not need to include the context_list in the prediction and they do not need to supply the llm_api_parameters. 

However, other metrics are requested that use llm guided evaluation (AnswerRelevance and Coherence). To get these metrics, the user needs to specify a client (openai or mistral), an api key and a model name, along with any other model kwargs. The api key can be stored in an environment variable or passed directly into model.evaluate_text_generation(). 

Each of these metrics will use API calls to the specified LLM service to get information relevant for computing the desired metrics. Some of these metrics, such as AnswerRelevance and Coherence, do not require any context, so can be used with a Q&A model that does not use context. Currently, none of these metrics use the context in their API calls or computations.

Note that AnswerRelevance is specific to the Q&A setting (RAG is a subcase of Q&A). AnswerRelevance measures how relevant the answer is to the question, by measuring the proportion of statements in the answer that are relevant to the question. This would not work as well in a less structured setting, such as summarization or content generation, where some statements in the generated text may not be directly relevant to the query. 

In [ ]:
import ast

In [ ]:
# Read in the dataset of queries, groundtruths and predictions. 
df = pd.read_csv("rag_data.csv")

In [ ]:
# For the purposes of this example, let's get metrics for just the first 5 datums.
MAX_DATUMS = 5

# Create, build and finalize the dataset and model.
dataset = Dataset.create(
    name="rag_dataset",
    metadata={
        "hf_dataset_name": "rag-datasets/mini_wikipedia",
        "hf_dataset_subset": "question-answer",
        "hf_dataset_split": "test",
        "shuffle_seed": 42,
        "number_of_records": 50,
    }
)
model = Model.create(
    name="rag_model",
    metadata={
        "embedding_model_name": "text-embedding-ada-002", # When we ran llama-index above, it defaulted to text-embedding-ada-002.
        "llm_model_name": "GPT3.5-turbo", # When we ran llama-index above, it defaulted to GPT3.5.
    }
)

# Create a list of datums
datum_list = []
for i in range(min(len(df), MAX_DATUMS)):
    row = df.iloc[i]

    datum_list.append(
        Datum(
            uid=f"query{i}",
            text=row["query"],
        )
    )

# Build and finalize the dataset
for i in range(min(len(df), MAX_DATUMS)):
    row = df.iloc[i]
    datum = datum_list[i]

    dataset.add_groundtruth(
        GroundTruth(
            datum=datum,
            annotations=[
                # Perhaps you have multiple correct or good groundtruth answers to the query.
                # The labels below are a trivial example, but you could have less trivial examples.
                # For example, to the query "When was the United States of America founded?", you might 
                # consider both "During the American Revolution" or "July 4th, 1776" to be good answers.
                Annotation(
                    text=row["groundtruth"],
                    metadata={"annotator": "Alice"},
                ),
                Annotation(
                    text="The answer is " + row["groundtruth"],
                    metadata={"annotator": "Bob"},
                ),
            ],
        )
    )
dataset.finalize()

# Build and finalize the model
for i in range(min(len(df), MAX_DATUMS)):
    row = df.iloc[i]
    datum = datum_list[i]

    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    text=row["prediction"],
                    context_list=ast.literal_eval(row["context_list"]),
                ),
            ],
        )
    )
model.finalize_inferences(dataset)

In [ ]:
# Using GPT4o to evaluate GPT3.5-turbo's predictions across a variety of metrics. 
eval_job = model.evaluate_text_generation(
    dataset,
    metrics_to_return=[
        "AnswerCorrectness",
        "AnswerRelevance", 
        "Bias",
        "BLEU", 
        "ContextPrecision",
        "ContextRecall",
        "ContextRelevance",
        "Faithfulness",
        "Hallucination",
        "ROUGE",
        "Toxicity",
    ],
    llm_api_params = {
        "client":"openai",
        "api_key":OPENAI_API_KEY,
        "data":{
            "model":"gpt-4o",
            "seed":2024,
        },
    },    
    metric_params={
        "BLEU": {
            "weights": [1, 0, 0, 0],
        }
    }
)

assert eval_job.wait_for_completion(timeout=3000) == EvaluationStatus.DONE

# These are the computed metrics.
eval_job.metrics

# Here are some example metrics. These are all for query0 and were evaluated by GPT-4o.
example_expected_metrics = [
    {
        'type': 'AnswerCorrectness',
        'value': 0.5,
        'parameters': {
            'dataset': 'rag_dataset', 
            'datum_uid': 'query0', 
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign.",
        },
    },
    {
        'type': 'AnswerRelevance',
        'value': 1.0,
        'parameters': {
            'dataset': 'rag_dataset',
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign."
        },
    },
    {
        'type': 'Bias',
        'value': 0.0,
        'parameters': {
            'dataset': 'rag_dataset',
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign."
        },
    },
    {
        'type': 'BLEU',
        'value': 0.10344827586206899,
        'parameters': {
            'dataset': 'rag_dataset',
            'weights': [1.0, 0.0, 0.0, 0.0],
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign."
        },
    },
    {
        'type': 'ContextPrecision', 
        'value': 0.0,
        'parameters': {
            'dataset': 'rag_dataset', 
            'datum_uid': 'query0', 
            'context_list': [
                "Cleveland was defeated in the 1888 presidential election, in part due to fraud (See Blocks of Five). He actually led in the popular vote over Benjamin Harrison (48.6% to 47.8%), but Harrison won the Electoral College by a 233-168 margin, largely by squeaking out a barely-over-1% win in Cleveland's home state of New York; in fact, had Cleveland won his home state, he would have won the electoral vote by a count of 204-197 (201 votes then needed for victory). Note, though, that Cleveland earned 24 of his electoral votes in states that he won by less than 1% (Connecticut, Virginia, and West Virginia).", 
                "Some of Cleveland's actions were controversial with political factions. Such criticisms include but are not limited to: his intervention in the Pullman Strike of 1894 in order to keep the railroads moving (a move which angered labor unions), his support of the gold standard, and opposition to free silver which alienated the agrarian wing of the Democrats.  Furthermore, critics complained that he had little imagination and seemed overwhelmed by the nation's economic disasters   depressions and strikes   in his second term. He lost control of his party to the agrarians and silverites in 1896."
            ]
        },
    },
    {
        'type': 'ContextRecall',
        'value': 0.0, 
        'parameters': {
            'dataset': 'rag_dataset', 
            'datum_uid': 'query0', 
            'context_list': [
                "Cleveland was defeated in the 1888 presidential election, in part due to fraud (See Blocks of Five). He actually led in the popular vote over Benjamin Harrison (48.6% to 47.8%), but Harrison won the Electoral College by a 233-168 margin, largely by squeaking out a barely-over-1% win in Cleveland's home state of New York; in fact, had Cleveland won his home state, he would have won the electoral vote by a count of 204-197 (201 votes then needed for victory). Note, though, that Cleveland earned 24 of his electoral votes in states that he won by less than 1% (Connecticut, Virginia, and West Virginia).", 
                "Some of Cleveland's actions were controversial with political factions. Such criticisms include but are not limited to: his intervention in the Pullman Strike of 1894 in order to keep the railroads moving (a move which angered labor unions), his support of the gold standard, and opposition to free silver which alienated the agrarian wing of the Democrats.  Furthermore, critics complained that he had little imagination and seemed overwhelmed by the nation's economic disasters   depressions and strikes   in his second term. He lost control of his party to the agrarians and silverites in 1896."
            ]
        },
    },
    {
        'type': 'ContextRelevance', 
        'value': 0.0,
        'parameters': {
            'dataset': 'rag_dataset', 
            'datum_uid': 'query0', 
            'context_list': [
                "Cleveland was defeated in the 1888 presidential election, in part due to fraud (See Blocks of Five). He actually led in the popular vote over Benjamin Harrison (48.6% to 47.8%), but Harrison won the Electoral College by a 233-168 margin, largely by squeaking out a barely-over-1% win in Cleveland's home state of New York; in fact, had Cleveland won his home state, he would have won the electoral vote by a count of 204-197 (201 votes then needed for victory). Note, though, that Cleveland earned 24 of his electoral votes in states that he won by less than 1% (Connecticut, Virginia, and West Virginia).", 
                "Some of Cleveland's actions were controversial with political factions. Such criticisms include but are not limited to: his intervention in the Pullman Strike of 1894 in order to keep the railroads moving (a move which angered labor unions), his support of the gold standard, and opposition to free silver which alienated the agrarian wing of the Democrats.  Furthermore, critics complained that he had little imagination and seemed overwhelmed by the nation's economic disasters   depressions and strikes   in his second term. He lost control of his party to the agrarians and silverites in 1896."
            ]
        },
    },
    # {
    #     'type': 'Coherence',
    #     'value': 4.0,
    #     'parameters': {
    #         'dataset': 'rag_dataset',
    #         'datum_uid': 'query0',
    #         'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign."
    #     },
    # },
    {
        'type': 'Faithfulness',
        'value': 0.0,
        'parameters': {
            'dataset': 'rag_dataset',
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign.",
            'context_list': [
                "Cleveland was defeated in the 1888 presidential election, in part due to fraud (See Blocks of Five). He actually led in the popular vote over Benjamin Harrison (48.6% to 47.8%), but Harrison won the Electoral College by a 233-168 margin, largely by squeaking out a barely-over-1% win in Cleveland's home state of New York; in fact, had Cleveland won his home state, he would have won the electoral vote by a count of 204-197 (201 votes then needed for victory). Note, though, that Cleveland earned 24 of his electoral votes in states that he won by less than 1% (Connecticut, Virginia, and West Virginia).",
                "Some of Cleveland's actions were controversial with political factions. Such criticisms include but are not limited to: his intervention in the Pullman Strike of 1894 in order to keep the railroads moving (a move which angered labor unions), his support of the gold standard, and opposition to free silver which alienated the agrarian wing of the Democrats.  Furthermore, critics complained that he had little imagination and seemed overwhelmed by the nation's economic disasters   depressions and strikes   in his second term. He lost control of his party to the agrarians and silverites in 1896."
            ],
        },
    },
    {
        'type': 'Hallucination',
        'value': 1.0,
        'parameters': {
            'dataset': 'rag_dataset',
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign.",
            'context_list': [
                "Cleveland was defeated in the 1888 presidential election, in part due to fraud (See Blocks of Five). He actually led in the popular vote over Benjamin Harrison (48.6% to 47.8%), but Harrison won the Electoral College by a 233-168 margin, largely by squeaking out a barely-over-1% win in Cleveland's home state of New York; in fact, had Cleveland won his home state, he would have won the electoral vote by a count of 204-197 (201 votes then needed for victory). Note, though, that Cleveland earned 24 of his electoral votes in states that he won by less than 1% (Connecticut, Virginia, and West Virginia).",
                "Some of Cleveland's actions were controversial with political factions. Such criticisms include but are not limited to: his intervention in the Pullman Strike of 1894 in order to keep the railroads moving (a move which angered labor unions), his support of the gold standard, and opposition to free silver which alienated the agrarian wing of the Democrats.  Furthermore, critics complained that he had little imagination and seemed overwhelmed by the nation's economic disasters   depressions and strikes   in his second term. He lost control of his party to the agrarians and silverites in 1896."
            ],
        },
    },
    {
        'type': 'ROUGE',
        'value': {
            'rouge1': 0.21052631578947364,
            'rouge2': 0.12121212121212122,
            'rougeL': 0.17142857142857143,
            'rougeLsum': 0.17142857142857143
        },
        'parameters': {
            'dataset': 'rag_dataset',
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign.",
            'rouge_types': ['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
            'use_stemmer': False
        },
    },
    {
        'type': 'Toxicity',
        'value': 0.0,
        'parameters': {
            'dataset': 'rag_dataset',
            'datum_uid': 'query0',
            'prediction': "Cleveland's opponents in 1884 criticized his alleged involvement in a scandal regarding an illegitimate child, which was used to counter his innocent image during the presidential campaign."
        },
    },
]

# Use Case #2: Summarization

## Load CNN Articles and get Summaries with GPT3.5-turbo

In [ ]:
import csv
import os
from datasets import load_dataset
import openai

openai_client = openai.OpenAI()

In [ ]:
# Load the cnn dataset. 
cnn_dataset = load_dataset("cnn_dailymail", "3.0.0")["test"]
cnn_dataset = cnn_dataset.shuffle(seed=42)
print(cnn_dataset)

In [ ]:
if os.path.exists("summarization_data.csv"):
    os.remove("summarization_data.csv")

NUMBER_OF_RECORDS = 50

instruction="You are a helpful assistant. Please summarize the following article in a few sentences."

with open("summarization_data.csv", mode="w") as data_file:
    data_writer = csv.writer(data_file, delimiter=",", quoting=csv.QUOTE_MINIMAL)
    data_writer.writerow(["text", "groundtruth", "prediction"])

    for i in range(NUMBER_OF_RECORDS):
        article = cnn_dataset[i]["article"]
        groundtruth = cnn_dataset[i]["highlights"]

        print(f"{i}: {groundtruth}")
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": article},
        ]

        response_object = openai_client.chat.completions.create(
            model="gpt-3.5-turbo", messages=messages, seed=42
        )
        prediction = response_object.choices[0].message.content

        print(f"prediction: {prediction}")
        data_writer.writerow([article, groundtruth, prediction])
    
    data_file.close()

## Evaluation in Valor

In this example, CNN articles are summarized by GPT3.5-turbo. Groundtruth summaries are also known for these articles. Both the datums (which contain the articles) and the groundtruths are added to the dataset. Then, the predictions are added to the model, which includes just the generated summary (there is not retrieved context for summarization).

The metrics requested are BLEU, ROUGE and Coherence. BLEU and ROUGE are used to measure the similarity between the generated summary and the groundtruth summary. Coherence is an llm-guided metric that measures the overall quality and cohesiveness of the generated summary.

In [2]:
# Read in the dataset of queries, groundtruths and predictions. 
df = pd.read_csv("summarization_data.csv")

In [5]:
print(df.iloc[46]['text'])
print(df.iloc[46]['prediction'])
print(df.iloc[47]['text'])
print(df.iloc[47]['prediction'])

Aston Villa take on Liverpool in their FA Cup semi-final encounter on Sunday with the competition both sides' last chance to win any silverware this season. Sportsmail columnist Jamie Redknapp looks ahead to the Wembley showdown and where the match could be won and lost with individual player duels. CHRISTIAN BENTEKE v MARTIN SKRTEL . This will be a heavyweight contest that could decide the game. Christian Benteke is superb in the air and Martin Skrtel will have his hands full. Liverpool have to stop the supply line because defending crosses has been their Achilles heel this season. Christian Benteke (centre) scored the only goal of the game as Villa won 1-0 at Tottenham on April 11 . Liverpool defender Martin Skrtel (right) will have his hands full trying to stop Benteke on Sunday afternoon . FABIAN DELPH v JORDAN HENDERSON . This should be a good contest between two England team-mates. Fabian Delph’s new deal was a real boost for Villa - he drives that midfield, though he doesn’t get

In [ ]:
# For the purpose of this example, let's get metrics for just the first 5 datums.
MAX_DATUMS = 5

# Create, build and finalize the dataset and model.
dataset = Dataset.create("summarization_dataset")
model = Model.create("summarization_model")

# Create a list of datums
datum_list = []
for i in range(min(len(df), MAX_DATUMS)):
    row = df.iloc[i]

    datum_list.append(
        Datum(
            uid=f"article{i}",
            text=row["text"],
            metadata={
                "query": "Summarize this article in a few sentences.", 
            }
        )
    )
    
# Build and finalize the dataset
for i in range(min(len(df), MAX_DATUMS)):
    row = df.iloc[i]
    datum = datum_list[i]

    dataset.add_groundtruth(
        GroundTruth(
            datum=datum,
            annotations=[
                Annotation(
                    text=row["groundtruth"],
                ),
            ],
        )
    )
dataset.finalize()

# Build and finalize the model
for i in range(min(len(df), MAX_DATUMS)):
    row = df.iloc[i]
    datum = datum_list[i]

    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    text=row["prediction"],
                )
            ],
        )
    )
model.finalize_inferences(dataset)

In [ ]:
# Using GPT4o to evaluate GPT3.5-turbo's predictions across a variety of metrics. 
eval_job = model.evaluate_text_generation(
    dataset,
    metrics_to_return=[
        "Bias",
        "BLEU",
        "Coherence",
        "ROUGE",
        "Toxicity",
    ],
    llm_api_params = {
        "client":"openai",
        "api_key":OPENAI_API_KEY,
        "data":{
            "model":"gpt-4o",
            "seed":2024,
        },
    },   
    metric_params={
        "BLEU": {
            "weights": [1, 0, 0, 0],
        }
    }
)

assert eval_job.wait_for_completion() == EvaluationStatus.DONE

eval_job.metrics

example_expected_metrics = [
    {
        'type': 'Bias',
        'value': 0.0,
        'parameters': {
            'dataset': 'summarization_dataset',
            'datum_uid': 'article4',
            'prediction': 'British taekwondo fighter Aaron Cook plans to compete for Moldova at the 2016 Olympics in Rio after being overlooked for the Great Britain squad in London 2012. Cook received funding from a Moldovan billionaire and has now obtained Moldovan citizenship. He has decided to no longer compete for Great Britain due to feeling overlooked and unsupported, and hopes to represent Moldova at international competitions, including the Olympics. The British Olympic Association could potentially block this move, as discussions are ongoing.'
        },
    },
    {
        'type': 'BLEU',
        'value': 0.3373493975903614,
        'parameters': {
            'dataset': 'summarization_dataset',
            'weights': [1.0, 0.0, 0.0, 0.0],
            'datum_uid': 'article4',
            'prediction': 'British taekwondo fighter Aaron Cook plans to compete for Moldova at the 2016 Olympics in Rio after being overlooked for the Great Britain squad in London 2012. Cook received funding from a Moldovan billionaire and has now obtained Moldovan citizenship. He has decided to no longer compete for Great Britain due to feeling overlooked and unsupported, and hopes to represent Moldova at international competitions, including the Olympics. The British Olympic Association could potentially block this move, as discussions are ongoing.'
        },
    },
    # {
    #     'type': 'Coherence',
    #     'value': 5.0,
    #     'parameters': {
    #         'dataset': 'summarization_dataset',
    #         'datum_uid': 'article4',
    #         'prediction': 'British taekwondo fighter Aaron Cook plans to compete for Moldova at the 2016 Olympics in Rio after being overlooked for the Great Britain squad in London 2012. Cook received funding from a Moldovan billionaire and has now obtained Moldovan citizenship. He has decided to no longer compete for Great Britain due to feeling overlooked and unsupported, and hopes to represent Moldova at international competitions, including the Olympics. The British Olympic Association could potentially block this move, as discussions are ongoing.'
    #     },
    # },
    {
        'type': 'ROUGE',
        'value': {
            'rouge1': 0.4915254237288136,
            'rouge2': 0.13793103448275862,
            'rougeL': 0.3389830508474576,
            'rougeLsum': 0.37288135593220345
        },
        'parameters': {
            'dataset': 'summarization_dataset',
            'datum_uid': 'article4',
            'prediction': 'British taekwondo fighter Aaron Cook plans to compete for Moldova at the 2016 Olympics in Rio after being overlooked for the Great Britain squad in London 2012. Cook received funding from a Moldovan billionaire and has now obtained Moldovan citizenship. He has decided to no longer compete for Great Britain due to feeling overlooked and unsupported, and hopes to represent Moldova at international competitions, including the Olympics. The British Olympic Association could potentially block this move, as discussions are ongoing.',
            'rouge_types': ['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
            'use_stemmer': False
        },
    },
    {
        'type': 'Toxicity',
        'value': 0.0,
        'parameters': {
            'dataset': 'summarization_dataset',
            'datum_uid': 'article4',
            'prediction': 'British taekwondo fighter Aaron Cook plans to compete for Moldova at the 2016 Olympics in Rio after being overlooked for the Great Britain squad in London 2012. Cook received funding from a Moldovan billionaire and has now obtained Moldovan citizenship. He has decided to no longer compete for Great Britain due to feeling overlooked and unsupported, and hopes to represent Moldova at international competitions, including the Olympics. The British Olympic Association could potentially block this move, as discussions are ongoing.'
        },
    },
]

# Use Case #3: Content Generation

## Some Example Content Generation Queries

In [ ]:
queries = [
    "Write about a haunted house from the perspective of the ghost.",
    "Explain to an elementary school student how to do long multiplication with the example 43 times 22. The resulting answer should be 946.",
    "Draft an email to a coworker explaining a project delay. Explain that the delay is due to funding cuts, which resulted in multiple employees being moved to different projects. Inform the coworker that the project deadline will have to be pushed back. Be apologetic and professional. Express eagerness to still complete the project as efficiently as possible.",
]

query_metadata = [
    {
        "request_type": "creative",
    },
    {
        "request_type": "educational",
    },
    {
        "request_type": "professional",
    },
]

In [ ]:
if os.path.exists("content_generation_data.csv"):
    os.remove("content_generation_data.csv")

instruction="You are a helpful assistant."

with open("content_generation_data.csv", mode="w") as data_file:
    data_writer = csv.writer(data_file, delimiter=",", quoting=csv.QUOTE_MINIMAL)
    data_writer.writerow(["query", "prediction"])

    for i in range(len(queries)):
        query = queries[i]

        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": query},
        ]
        response_object = client.chat.completions.create(
            model="gpt-3.5-turbo", messages=messages, seed=42
        )
        prediction = response_object.choices[0].message.content

        print(f"prediction: {prediction}")
        data_writer.writerow([query, prediction])
    
    data_file.close()

## Evaluation in Valor

In this example, text is generated in response to multiple open-ended queries. These queries are written so that there is no correct or canonically good response, so there are no groundtruth annotations for these queries. To build the dataset, we add the queries to the datums, then add groundtruths to the dataset that only contain the datums and no annotations. We add the generated text as predictions to the model. 

The only metric we request is Coherence, which is an llm-guided metric that measures the overall quality and cohesiveness of the generated text. We don't use text comparison metrics as there are no groundtruth annotations to compare to. We don't use Q&A/RAG metrics as there is no context and no query-answer structure to the data.

In [ ]:
# Read in the dataset of queries and predictions.
df = pd.read_csv("content_generation_data.csv")

In [ ]:
# Create, build and finalize the dataset and model.
dataset = Dataset.create("content_generation_dataset")
model = Model.create("content_generation_model")

# Create a list of datums
datum_list = []
for i in range(len(df)):
    row = df.iloc[i]

    datum_list.append(
        Datum(
            uid=f"query{i}",
            text=row["query"],
        )
    )

# Build and finalize the dataset
for i in range(len(df)):
    row = df.iloc[i]
    datum = datum_list[i]

    # There are no groundtruth annotations for content generation.
    dataset.add_groundtruth(
        GroundTruth(
            datum=datum,
            annotations=[],
        )
    )
dataset.finalize()

# Build and finalize the model
for i in range(len(df)):
    row = df.iloc[i]
    datum = datum_list[i]

    model.add_prediction(
        dataset, 
        Prediction(
            datum=datum,
            annotations=[
                Annotation(
                    text=row["prediction"],
                )
            ],
        )
    )
model.finalize_inferences(dataset)

In [ ]:
# Using GPT4o to evaluate GPT3.5-turbo's predictions across a variety of metrics. 
eval_job = model.evaluate_text_generation(
    dataset,
    metrics_to_return=[
        "Bias",
        "Toxicity",
    ],
    llm_api_params = {
        "client":"openai",
        "api_key":OPENAI_API_KEY,
        "data":{
            "model":"gpt-4o",
            "seed":2024,
        },
    },
)

assert eval_job.wait_for_completion() == EvaluationStatus.DONE

eval_job.metrics

example_expected_metrics = [
    {
        'type': 'Bias',
        'value': 0.0,
        'parameters': {
            'dataset': 'content_generation_dataset',
            'datum_uid': 'query2',
            'prediction': """Subject: Project Delay Due to Funding Cuts

Dear [Coworker's Name],

I hope this message finds you well. I am writing to update you on the status of our project and unfortunately, convey some disappointing news regarding a delay in its completion.

Due to recent funding cuts within our department, our project team has been significantly affected. Several team members, including myself, have been relocated to work on other projects to address the shifting priorities resulting from the budget constraints.

As a consequence of these unexpected changes, it is with regret that I must inform you that the original deadline for our project will need to be extended. I understand the inconvenience that this may cause, and I sincerely apologize for any inconvenience this delay may bring to you and your plans.

Rest assured that despite this setback, I am fully committed to ensuring that we still deliver the project with utmost efficiency and quality. I am exploring all possible avenues to mitigate the delay and work towards completing our project in a timely manner.

I appreciate your understanding and patience during this challenging time. Your ongoing support and collaboration are invaluable as we navigate through this situation together. If you have any concerns or questions, please do not hesitate to reach out to me.

Thank you for your understanding, and I look forward to working with you to successfully finalize our project.

Warm regards,

[Your Name]"""
        },
    },
#     {
#         "value": 5.0,
#         "type": "Coherence",
#         "parameters": {
#             "dataset": "content_generation_dataset",
#             "datum_uid": "query2",
#             "prediction": """Subject: Project Delay Due to Funding Cuts

# Dear [Coworker's Name],

# I hope this message finds you well. I am writing to update you on the status of our project and unfortunately, convey some disappointing news regarding a delay in its completion.

# Due to recent funding cuts within our department, our project team has been significantly affected. Several team members, including myself, have been relocated to work on other projects to address the shifting priorities resulting from the budget constraints.

# As a consequence of these unexpected changes, it is with regret that I must inform you that the original deadline for our project will need to be extended. I understand the inconvenience that this may cause, and I sincerely apologize for any inconvenience this delay may bring to you and your plans.

# Rest assured that despite this setback, I am fully committed to ensuring that we still deliver the project with utmost efficiency and quality. I am exploring all possible avenues to mitigate the delay and work towards completing our project in a timely manner.

# I appreciate your understanding and patience during this challenging time. Your ongoing support and collaboration are invaluable as we navigate through this situation together. If you have any concerns or questions, please do not hesitate to reach out to me.

# Thank you for your understanding, and I look forward to working with you to successfully finalize our project.

# Warm regards,

# [Your Name]""",
#         },
#     },
    {
        'type': 'Toxicity',
        'value': 0.0,
        'parameters': {
            'dataset': 'content_generation_dataset',
            'datum_uid': 'query2',
            'prediction': """Subject: Project Delay Due to Funding Cuts

Dear [Coworker's Name],

I hope this message finds you well. I am writing to update you on the status of our project and unfortunately, convey some disappointing news regarding a delay in its completion.

Due to recent funding cuts within our department, our project team has been significantly affected. Several team members, including myself, have been relocated to work on other projects to address the shifting priorities resulting from the budget constraints.

As a consequence of these unexpected changes, it is with regret that I must inform you that the original deadline for our project will need to be extended. I understand the inconvenience that this may cause, and I sincerely apologize for any inconvenience this delay may bring to you and your plans.

Rest assured that despite this setback, I am fully committed to ensuring that we still deliver the project with utmost efficiency and quality. I am exploring all possible avenues to mitigate the delay and work towards completing our project in a timely manner.

I appreciate your understanding and patience during this challenging time. Your ongoing support and collaboration are invaluable as we navigate through this situation together. If you have any concerns or questions, please do not hesitate to reach out to me.

Thank you for your understanding, and I look forward to working with you to successfully finalize our project.

Warm regards,

[Your Name]"""
        },
    },
]    